In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import json
import matplotlib.pyplot as plt

In [2]:
raw_movies = pd.read_csv('./data/head_10_movies.csv').head(n=10)
raw_credits = pd.read_csv('./data/head_10_credits.csv').head(n=10)

### Remove unwanted credit features

In [3]:
unwanted_credit_features = ['title']
credits = raw_credits[raw_credits.columns.difference(unwanted_credit_features)]

In [4]:
movies = pd.merge(left=raw_movies, right=credits, left_on='id', right_on='movie_id')

### Turn json into python objects

In [5]:
json_columns = ['genres', 'keywords', 'production_countries', 'spoken_languages', 'cast', 'crew']
for column in json_columns:
    movies[column] = movies[column].apply(json.loads, encoding="utf-8")

In [6]:
### Turn actors list into features (a bunch of columns)

In [7]:
movies_actors_ids = []
for actors in movies.cast:
    movie_actors_ids = []
    for actor in actors:
        movie_actors_ids.append(str(actor['id']))
    movies_actors_ids.append(' '.join(movie_actors_ids))

    

In [8]:
movies['actor_ids'] = movies_actors_ids
cleaned = movies.set_index('id').actor_ids.str.split(' ', expand=True).stack()
actor_features = pd.get_dummies(cleaned, prefix='actor').groupby(level=0).sum()

In [9]:
actor_features['id'] = actor_features.index

In [10]:
## movies = pd.merge(left=movies, right=actor_features, left_on='id', right_on='id')

In [11]:
#movies.describe()

In [21]:
unimportant_features = [
    'genres',
    'homepage',
    'keywords',
    'original_language',
    'original_title',
    'overview',
    'production_companies',
    'spoken_languages',
    'status',
    'tagline',
    'title',
    'cast',
    'crew',
    'movie_id',
    'actor_ids'
]
important_features = [
    'id',
    'budget',
    'popularity',
    'vote_average',
    'vote_count'
]

movies_with_important_features = movies[important_features]

In [34]:
y = movies.popularity.astype('int')
x = pd.merge(left=movies_with_important_features, right=actor_features, left_on='id', right_on='id')

In [35]:
y

0    150
1    139
2    107
3    112
4     43
5    115
6     48
7    134
8     98
9    155
Name: popularity, dtype: int64

In [36]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(x,y)

GaussianNB(priors=None)

In [37]:
clf.predict(x)

array([150, 139, 107, 112,  43, 115,  48, 134,  98, 155])

In [ ]:
# movies = pd.DataFrame({'revenue': [100], 'actors': 'john sam'})

In [ ]:
# movies

In [ ]:
x.dtypes

In [ ]:
x